In [1]:
from keras.preprocessing.image import load_img, img_to_array
from scipy.misc import imsave
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time

from keras.applications import vgg16
from keras import backend as K

Using TensorFlow backend.


In [2]:
base_image_path = 'image.jpg'
style_reference_image_path = 'style.jpg'
result_prefix = 'im'
iterations = 10

# these are the weights of the different loss components
total_variation_weight = 1.0
style_weight = 0.7
content_weight = 0.5

# dimensions of the generated picture.
width, height = load_img(base_image_path).size
img_nrows = 400
img_ncols = int(width * img_nrows / height)

In [3]:
# util function to open, resize and format pictures into appropriate tensors
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_nrows, img_ncols))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg16.preprocess_input(img)
    return img

# util function to convert a tensor into a valid image
def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, img_nrows, img_ncols))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [4]:
# get tensor representations of our images
base_image = K.variable(preprocess_image(base_image_path)) #K.variable create tensor images
style_reference_image = K.variable(preprocess_image(style_reference_image_path))

In [5]:
# this will contain our generated image
combination_image = K.placeholder((1, img_nrows, img_ncols, 3))

# combine the 3 images into a single Keras tensor
input_tensor = K.concatenate([base_image,
                              style_reference_image,
                              combination_image], axis=0)

In [6]:
# build the VGG16 network with our 3 images as input
# the model will be loaded with pre-trained ImageNet weights
model = vgg16.VGG16(input_tensor=input_tensor,
                    weights='imagenet', include_top=False)
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (3, 400, 533, 3)      0                                            
____________________________________________________________________________________________________
block1_conv1 (Convolution2D)     (3, 400, 533, 64)     1792        input_1[0][0]                    
____________________________________________________________________________________________________
block1_conv2 (Convolution2D)     (3, 400, 533, 64)     36928       block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_pool (MaxPooling2D)       (3, 200, 266, 64)     0           block1_conv2[0][0]               
___________________________________________________________________________________________

In [7]:
# get the symbolic outputs of each "key" layer (we gave them unique names).
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
for key in outputs_dict:
    print key, outputs_dict[key]

block4_pool Tensor("MaxPool_3:0", shape=(3, 25, 33, 512), dtype=float32)
block1_pool Tensor("MaxPool:0", shape=(3, 200, 266, 64), dtype=float32)
block4_conv1 Tensor("Relu_7:0", shape=(3, 50, 66, 512), dtype=float32)
block2_conv1 Tensor("Relu_2:0", shape=(3, 200, 266, 128), dtype=float32)
block2_conv2 Tensor("Relu_3:0", shape=(3, 200, 266, 128), dtype=float32)
block4_conv2 Tensor("Relu_8:0", shape=(3, 50, 66, 512), dtype=float32)
block4_conv3 Tensor("Relu_9:0", shape=(3, 50, 66, 512), dtype=float32)
block2_pool Tensor("MaxPool_1:0", shape=(3, 100, 133, 128), dtype=float32)
block5_conv3 Tensor("Relu_12:0", shape=(3, 25, 33, 512), dtype=float32)
block5_conv2 Tensor("Relu_11:0", shape=(3, 25, 33, 512), dtype=float32)
block5_conv1 Tensor("Relu_10:0", shape=(3, 25, 33, 512), dtype=float32)
block3_pool Tensor("MaxPool_2:0", shape=(3, 50, 66, 256), dtype=float32)
block1_conv2 Tensor("Relu_1:0", shape=(3, 400, 533, 64), dtype=float32)
block1_conv1 Tensor("Relu:0", shape=(3, 400, 533, 64), dtype

In [8]:

# This is achieved through the optimization of a loss function
# that has 3 components: "style loss", "content loss",
# and "total variation loss":

# - The total variation loss imposes local spatial continuity between
# the pixels of the combination image, giving it visual coherence.

# - The style loss is where the deep learning keeps in --that one is defined
# using a deep convolutional neural network. Precisely, it consists in a sum of
# L2 distances between the Gram matrices of the representations of
# the base image and the style reference image, extracted from
# different layers of a convnet (trained on ImageNet). The general idea
# is to capture color/texture information at different spatial
# scales (fairly large scales --defined by the depth of the layer considered).

#  - The content loss is a L2 distance between the features of the base
# image (extracted from a deep layer) and the features of the combination image,
# keeping the generated image close enough to the original one.








# the gram matrix of an image tensor (feature-wise outer product)
def gram_matrix(x):
#features = K.batch_flatten(x)
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

# the "style loss" is designed to maintain
# the style of the reference image in the generated image.
# It is based on the gram matrices (which capture style) of
# feature maps from the style reference image
# and from the generated image
def style_loss(style, combination):
    assert K.ndim(style) == 3
    assert K.ndim(combination) == 3
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))


# an auxiliary loss function
# designed to maintain the "content" of the
# base image in the generated image
def content_loss(base, combination):
    return K.sum(K.square(combination - base))

# the 3rd loss function, total variation loss,
# designed to keep the generated image locally coherent, coherent meaning -(logical and consistent).
def total_variation_loss(x):
    assert K.ndim(x) == 4
    
    a = K.square(x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
    b = K.square(x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

In [9]:
# combine these loss functions into a single scalar
loss = K.variable(0.)
layer_features = outputs_dict['block4_conv2']
base_image_features = layer_features[0, :, :, :] # 1 for style_image_features
combination_features = layer_features[2, :, :, :]
loss += content_weight * content_loss(base_image_features,
                                      combination_features)

In [10]:
feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']

In [11]:
for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss += (style_weight / len(feature_layers)) * sl

loss += total_variation_weight * total_variation_loss(combination_image)

In [12]:
# get the gradients of the generated image wrt the loss
grads = K.gradients(loss, combination_image)

outputs = [loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

f_outputs = K.function([combination_image], outputs)

In [13]:
def eval_loss_and_grads(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((1, 3, img_nrows, img_ncols))
    else:
        x = x.reshape((1, img_nrows, img_ncols, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

In [18]:
class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

evaluator = Evaluator()

In [20]:
# run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss
if K.image_data_format() == 'channels_first':
    x = np.random.uniform(0, 255, (1, 3, img_nrows, img_ncols)) - 128.
else:
    x = np.random.uniform(0, 255, (1, img_nrows, img_ncols, 3)) - 128.

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    
    img = deprocess_image(x.copy())
    fname = 'results/im' + '_at_iteration_%d.png' % i
    imsave(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

AttributeError: 'module' object has no attribute 'image_data_format'